In [1]:
import tensorflow as tf
from tensorflow.contrib.keras.python.keras import backend as K
from tensorflow.contrib.keras.python.keras.models import load_model, Sequential

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def
from datetime import datetime as dt



In [2]:
sess = tf.Session()
K.set_session(sess)

# disable loading of learning nodes
K.set_learning_phase(0)


tdatetime = dt.now()
tstr = tdatetime.strftime('%Y%m%d%H%M%S')

# load model
model = load_model("../output/cnn-dog-cat.h5")
config = model.get_config()
weights = model.get_weights()
new_Model = Sequential.from_config(config)
new_Model.set_weights(weights)

# export current best model
export_path = "../output/tensorflow_model/" + tstr
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'accelerations': new_Model.input},
                                  outputs={'scores': new_Model.output})


In [3]:
with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={
                                             signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature})
    print(signature)
    builder.save()


INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
inputs {
  key: "inputs"
  value {
    name: "conv2d_1_input_1:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 64
      }
      dim {
        size: 64
      }
      dim {
        size: 3
      }
    }
  }
}
outputs {
  key: "outputs"
  value {
    name: "activation_5_1/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 2
      }
    }
  }
}
method_name: "tensorflow/serving/predict"

INFO:tensorflow:SavedModel written to: b'../output/tensorflow_model/20170610164832/saved_model.pb'
